In [2]:
### THIS SCRIPT COLLECTS DATA FROM SEP and InPhO ### 

In [3]:
# import dependencies
import time
import pymongo
import re
import json
import requests

from tqdm import tqdm
from pprint import pprint
from pathlib import Path
from bs4 import BeautifulSoup

#import local libraries
import lib_sepinpho as sep
import lib_fileops as io

In [4]:
##### INIT GLOBAL VARIABLES#####

#init Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#connect to database
db = client.visualizing_sep

In [8]:
### SCRAPE TOC FROM WINTER 2019 SEP ARCHIVE ###
### Date Completed: 3.30.2020 ###

#set variable options for Winter 2019 download
toc_collection = db.toc_2019_winter
sep_edition = 'Winter_2019'
search_url = 'https://plato.stanford.edu/archives/win2019/contents.html'
base_url = 'https://plato.stanford.edu/archives/win2019/'
save_as_html = '../downloads/tocs/toc_2019_winter_all.html'

# print(f'Starting: SEP {sep_edition} TOC Scrape\n')

# #get list of elements to save into toc_collection 
# toc_elements = sep.scrape_sep_toc(sep_edition, search_url, base_url, save_as_html)

# #update MonogDB collection
# toc_collection.insert_many(toc_elements)

# print(f'Success! Completed SEP {sep_edition} TOC Scrape\n')

Starting: SEP Winter_2019 TOC Scrape

Success! Completed SEP Winter_2019 TOC Scrape



In [ ]:
### DOWNLOAD WINTER 2019 SEP ARCHIVE, AND STORE THEM LOCALLY###
# Winter 2019 Archive: Completed 1/30/2020. This took 7 hours, so don't do it again!
# Updated the code on 4.10.2020 to make the download process cleaner, but the files were NOT re-downloaded

#set variable options for Winter 2019 download
toc_collection = db.toc_2019_winter
toc_pages = list(toc_collection.find().sort('link_url'))
save_as_directory = '..downloads/articles/sep_2019_all/'

#download SEP pages from archive
#sep.download_sep_pages(toc_pages,save_as_directory)

In [9]:
### SCRAPE WHATS NEW TOC FROM SPRING 2020 SEP ARCHIVE ###
### Downloaded Spring 2020 TOC updates: completed 4.10.2020 ###

#set variable options for Spring 2020 download
toc_collection = db.toc_2020_spring_new
sep_edition = 'Spring_2020'
search_url = 'https://plato.stanford.edu/archives/spr2020/new.html'
base_url = 'https://plato.stanford.edu/archives/spr2020/'
save_as_html = '../downloads/tocs/toc_2020_spring_new.html'

print(f'Starting: SEP {sep_edition} TOC Scrape\n')

toc_elements = sep.scrape_sep_toc(sep_edition, search_url, base_url, save_as_html)
toc_collection.insert_many(toc_elements)

print(f'Success! Completed SEP {sep_edition} TOC Scrape\n')


Starting: SEP Spring_2020 TOC Scrape

Success! Completed SEP Spring_2020 TOC Scrape



In [4]:
### DOWNLOAD SPRING 2020 WHATS NEW FILES, AND STORE THEM LOCALLY###
#Spring 2020 Archive: Completed 4.10.2020#

#set variable options for Spring 2020 download
toc_collection = db.toc_2020_spring_new
toc_pages = list(toc_collection.find().sort('link_url'))
save_as_directory = '../downloads/articles/sep_2020_spring/'

#download SEP pages from archive
sep.download_sep_pages(toc_pages,save_as_directory)

Processing: 100%|██████████| 67/67 [15:53<00:00, 14.23s/it]


In [10]:
### SCRAPE WHATS NEW TOC FROM SUMMER 2020 SEP ARCHIVE ###
### Downloaded Summer 2020 TOC updates: completed 7.13.2020 ###

#set variable options for Summer 2020 download
toc_collection = db.toc_2020_summer_new
sep_edition = 'Summer_2020'
search_url = 'https://plato.stanford.edu/archives/sum2020/new.html'
base_url = 'https://plato.stanford.edu/archives/sum2020/'
save_as_html = '../downloads/tocs/toc_2020_summer_new.html'

print(f'Starting: SEP {sep_edition} TOC Scrape\n')

toc_elements = sep.scrape_sep_toc(sep_edition, search_url, base_url, save_as_html)
toc_collection.insert_many(toc_elements)

print(f'Success! Completed SEP {sep_edition} TOC Scrape\n')


Starting: SEP Summer_2020 TOC Scrape

Success! Completed SEP Summer_2020 TOC Scrape



In [4]:
### DOWNLOAD SUMMER 2020 WHATS NEW FILES, AND STORE THEM LOCALLY###
#Summer 2020 Archive: Completed 7.13.2020#

#set variable options for Summer 2020 download
toc_collection = db.toc_2020_summer_new
toc_pages = list(toc_collection.find().sort('link_url'))
save_as_directory = '../downloads/articles/sep_2020_summer/'

#download SEP pages from archive
sep.download_sep_pages(toc_pages,save_as_directory)

Processing: 0it [00:00, ?it/s]


In [4]:
### PARSING LOCAL SEP FILES ###

### After downloading the Winter 2019 archive and the Spring 2020 updates, I moved the updates into the same folder, and then ran the parsing script on all of the files at once. This replaced the prior verion of the parsed data objects in MongoDB. 
###

#set variable options for parsing
collection_to_update = db.sep_entries
file_path='../downloads/articles/sep_2019_all'
file_type='*.html'
sep_files = io.get_local_files(file_path,file_type)
for sep_file in tqdm(sep_files):
    try:
        sep_object = sep.parse_sep_file(sep_file)
        collection_to_update.insert_one(sep_object)
    except:
        print(f'Failed at: @{sep_file.name}')
    

n
 89%|████████▉ | 1499/1680 [42:25<04:24,  1.46s/it]Error: No InPhO entry
http://inpho.cogs.indiana.edu/idea/1225.json
 92%|█████████▏| 1547/1680 [43:41<03:14,  1.46s/it]Error: No InPhO entry
http://inpho.cogs.indiana.edu/idea/5619.json
100%|██████████| 1680/1680 [47:13<00:00,  1.69s/it]


In [3]:
#Update collection when new files downloaded

collection_to_update = db.sep_entries
file_path='../downloads/articles/sep_2020_summer'
file_type='*.html'
sep_files = io.get_local_files(file_path,file_type)
for sep_file in tqdm(sep_files):
    sep_object = sep.parse_sep_file(sep_file)
    update_string = sep.update_main_document_data(sep_object)
    find = {"page_url": sep_object['page_url']}
    collection_to_update.update_one(find, {'$set': update_string}, upsert=True)



 15%|█▍        | 13/88 [00:21<01:37,  1.30s/it]Error: No InPhO entry
http://inpho.cogs.indiana.edu/thinker/2844.json
 31%|███       | 27/88 [00:46<01:38,  1.61s/it]Error: No InPhO entry
http://inpho.cogs.indiana.edu/idea/1626.json
 35%|███▌      | 31/88 [00:52<01:25,  1.51s/it]Error: No InPhO entry
http://inpho.cogs.indiana.edu/idea/1467.json
 41%|████      | 36/88 [00:59<01:08,  1.32s/it]Error: No InPhO entry
http://inpho.cogs.indiana.edu/idea/1864.json
 49%|████▉     | 43/88 [01:10<01:03,  1.42s/it]Error: No InPhO entry
http://inpho.cogs.indiana.edu/idea/6354.json
 90%|████████▉ | 79/88 [02:10<00:13,  1.46s/it]Error: No InPhO entry
http://inpho.cogs.indiana.edu/thinker/3936.json
 95%|█████████▌| 84/88 [02:19<00:06,  1.52s/it]Error: No InPhO entry
http://inpho.cogs.indiana.edu/idea/5646.json
100%|██████████| 88/88 [02:24<00:00,  1.64s/it]


In [4]:
db.sep_entries.create_index('title')

'title_1'

In [11]:
#add/update authors 

toc_collection = db.toc_2019_winter
collection_to_update = db.sep_entries

toc_entries = list(toc_collection.find({}))
print(len(toc_entries))
# for entry in tqdm(toc_entries):
#     sep.add_author(entry['link_url'],entry['authors'], collection_to_update)

1671
